This notebook does the following:

1. **Retrieve Nucleotide Accession IDs and GFF Files**

2. **Download GFF3 Files**
   - Use a script to download GFF3 files from NCBI

3. **Find Overlap Between Genes in EggNOG and GFF Files**
   - Define functions to process files and extract unique values.
   - Process gene groups to find corresponding GFF files and extract gene features.
   - Write results to JSON and log skipped taxa.

4. **Download and Prune ASTRAL WoL Tree**
   - Prune the tree to include only taxa with gene data.
   - Replace leaf names with taxon IDs and write pruned trees to files.

5. **Extract Gene Trees of Interest**
   - Extract gene trees based on the number of genes and taxa.

6. **Prune Gene Trees**

7. **Check Taxa Consistency**
   - Ensure taxa in WoL tree and gene trees are consistent.
   - Prune WoL tree variants to include only taxa in gene trees.

8. **Download Genome Sequence Files**
   - Download genome sequence files for each taxon in the dataset.

9. **Replace Leaf Labels in Gene Trees**
   - Replace the first underscore with a dot in leaf labels of gene trees.

10. **Create Gene Features TSV**
    - Extract gene features for the subset of genes in the gene trees.

11. **Create Presence-Absence Matrices**
    - Create presence-absence matrices for GLOOME and COUNT.
    - Write matrices to TSV and FASTA files.

12. **Extract Function Profiles**
    - Extract function profiles for NOGs in the dataset from the EggNOG database.

In [1]:
# we find a subset of the data in EGGNOG, for which we can retrieve nucleotide data
# this means that
# 1. first we find for which genomes in EggNOG can we find NCBI genome assembly accession IDs
# 2. We download the assembly gff files for these acc IDs
# 3. We look through the list of genes in EggNOG, which we can find in the gff files.
taxonomicID = '1236'
data_dir = '../data/'

## Retrieve Nucleotide Accession IDs and GFF files

In [2]:
%%bash -s "$taxonomicID" "$data_dir"
## extract data for this dataset based on taxonomic ID declared before, using ripgrep
rg -w "^$1" $2/eggnog6/e6.og2seqs_and_species.tsv > $2$1_og2seqs_and_species.tsv

In [3]:
%%bash -s "$taxonomicID" "$data_dir"
# make a list of all taxa in *_og2seqs_and_species.tsv and write it to *_all.taxa
# the fifth column in this TSV file is a comma-delimited string of taxon IDs
awk -F"\t" '{print $5}' $2$1_og2seqs_and_species.tsv | tr ',' '\n' | sort -n | uniq > $2$1_all.taxa

Use `src/download_ncbi_genome_sequences.py` to download the gff3 files from NCBI, using multiprocessing. The script will download the files in the `data_dir`
    
```bash
nohup ~/mambaforge/envs/hgt_analyses/bin/python src/download_ncbi_genome_sequences.py -i 1236_all.taxa --to-download gff3  > nohup_gff3_dload.log & disown
```

## Find overlap between genes in EggNOG and genes in the GFF files

In [4]:
# suppress SyntaxWarnings in ete3 import using the warnings module
import warnings
warnings.filterwarnings("ignore", category=SyntaxWarning)

In [5]:
import csv
import sys
import pandas as pd
import multiprocessing as mp
import gffutils
import os
import json
import tqdm
import csv
import ete3
import numpy as np

In [6]:

csv.field_size_limit(sys.maxsize)

all_taxa_filepath = f'{data_dir}{taxonomicID}_all.taxa'
nog_members_tsv_filepath = f'{data_dir}{taxonomicID}_og2seqs_and_species.tsv'


def process_file(input_file_path, column_index):
    # This set will store all unique values from the specified column
    all_values = set()

    # Open the file and read it line by line
    with open(input_file_path, 'r') as fo:
        reader = csv.reader(fo, delimiter='\t')
        for row in reader:
            # Split the column by commas and add the items to the set
            if row[column_index]:  # Ensure the column isn't empty
                values = row[column_index].split(',')
                all_values.update(value.strip() for value in values)

    return all_values


unique_values = process_file(nog_members_tsv_filepath, 5)

# Write the unique values to a file or process them further
with open(f'{data_dir}{taxonomicID}_all.genes', 'w') as out_file:
    out_file.write('\n'.join(sorted(unique_values)) + '\n')

In [7]:
# read in the values from the *all.genes file into a df with a single column
all_genes_filepath = data_dir + taxonomicID + '_all.genes'

all_genes_df = pd.read_csv(
    all_genes_filepath, header=None, names=['locus_tag'])

taxon_acc_id_filepath = f'{data_dir}/genome_data/{taxonomicID}_all_accession_ids.tsv'
genome_data_dir = f'{data_dir}/genome_data/'

taxon_acc_id_df = pd.read_csv(taxon_acc_id_filepath, sep='\t', header=0)
taxon_acc_id_df['taxon_id'] = taxon_acc_id_df['taxon_id'].astype(str)

# add a column called "taxon_id" based on the first element in the period delimited 'locus_tag' column string
all_genes_df['taxon_id'] = all_genes_df['locus_tag'].apply(
    lambda x: x.split('.', 1)[0]).astype(str)

# groupby taxon and for every taxon, run this fn
# this fn takes finds the corresponding gff file using the taxon to gff mapping in taxon_acc_id_df
# it then looks for the locus_tag values in the gff. It returns a list of all locus_tag values that exist for this taxon in the gff file


def process_gene_group(gene_group, taxon_acc_id_df, genome_data_dir):
    """
    This function takes a group of genes from the all_genes_df dataframe and processes the corresponding gff file.
    It returns a list of gene IDs that are found in the gff file.
    """
    taxon_id = gene_group['taxon_id'].iloc[0]
    locus_tags = set(gene_group['locus_tag'])
    # remove taxon_id from locus_tags (i.e. the first element in the period delimited 'locus_tag' column string)
    locus_tags = {locus_tag.split('.', 1)[1] for locus_tag in locus_tags}

    # find corresponding gff file for taxon ID
    if taxon_acc_id_df['taxon_id'].isin([taxon_id]).any():
        acc_id = taxon_acc_id_df[taxon_acc_id_df['taxon_id']
                                 == taxon_id]['acc_id'].values[0]
    else:
        # print(f'taxon ID {taxon_id} not found. Skipping')
        return 'skipped: taxon ID not in TSV', taxon_id
    gff_filepath = os.path.join(genome_data_dir, f'{taxon_id}_{acc_id}_genomic.gff')
    if not os.path.exists(gff_filepath):
        # print(f'gff file {gff_filepath} not found. Skipping')
        return 'skipped: gff file not found', taxon_id

    # create gffutils df from the gff file
    gff_db = gffutils.create_db(gff_filepath, ':memory:', merge_strategy='merge')


    # return the list of dict of locus_tag values found in the gff file, the taxon_id, the acc_id, and the gene_features_list
    return_dict = {}

    # iterate over each feature of type 'gene' in the gff_db
    for feature in gff_db.features_of_type('gene'):
        # check if 'locus_tag' or 'old_locus_tag' is in feature.attributes and if its value is in locus_tags
        if 'locus_tag' in feature.attributes and feature.attributes['locus_tag'][0] in locus_tags:
            # if yes, append the locus_tag and the entire feature as a dict, including information of start, end, strand, and feature attributes
            return_dict[feature.attributes['locus_tag'][0]] = {
                'id': feature.id,  # 'gene:NC_000913:thrL'
                'locus_tag': feature.attributes['locus_tag'][0],
                'start': feature.start,
                'end': feature.end,
                'strand': feature.strand,
                'attributes': dict(feature.attributes),
                'seqid': feature.seqid
            }

        elif 'old_locus_tag' in feature.attributes and feature.attributes['old_locus_tag'][0] in locus_tags:
            # if yes, append to dict
            return_dict[feature.attributes['old_locus_tag'][0]] = {
                'id': feature.id,
                'locus_tag': feature.attributes['old_locus_tag'][0],
                'start': feature.start,
                'end': feature.end,
                'strand': feature.strand,
                'attributes': dict(feature.attributes),
                'seqid': feature.seqid
            }
            
    # if fewer than 10 genes are found, skip
    if len(return_dict) < 10:
        return 'skipped: less than 10 genes found', taxon_id

    return 'values', taxon_id, acc_id, return_dict


def process_gene_group_wrapper(args):
    return process_gene_group(*args)

# groupby taxon_id the genes in all_genes_df
gene_groups = all_genes_df.groupby('taxon_id')

# prepare arguments for process_gene_group function
args = [(group, taxon_acc_id_df, genome_data_dir) for _, group in gene_groups]

taxa_acc_genes_dict = {}
skipped_taxa_list = []
# create a multiprocessing Pool
with mp.Pool(mp.cpu_count()) as pool:
    # apply process_gene_group function to each group in parallel
    for pool_return in tqdm.tqdm(pool.imap_unordered(process_gene_group_wrapper, args), total=len(args)):
        if pool_return[0].startswith('skipped'):
            skipped_taxa_list.append((pool_return[1], pool_return[0]))
        elif pool_return[0] == 'values':
            taxa_acc_genes_dict[pool_return[1]] = {
                'acc_id': pool_return[2], 'genes': pool_return[3]}
        else:
            print('ERROR: invalid return value')
            break

# write the results to a json file
with open(f'{data_dir}{taxonomicID}_gene_features.json', 'w') as out_file:
    json.dump(taxa_acc_genes_dict, out_file, indent=4)
# log number of taxa in this json
print(f'Number of taxa with gene data: {len(taxa_acc_genes_dict)}')

# write the skipped taxa to a file
with open(f'{data_dir}{taxonomicID}_skipped_taxa.tsv', 'w') as out_file:
    # csv with quotes
    writer = csv.writer(out_file, quoting=csv.QUOTE_MINIMAL, delimiter='\t')
    writer.writerows(skipped_taxa_list)

100%|██████████| 1548/1548 [00:56<00:00, 27.46it/s]


Number of taxa with gene data: 853


In [8]:
# find which locus_tags are present in the json file of gene_features
gene_features_filepath = f'{data_dir}{taxonomicID}_gene_features.json'
with open(gene_features_filepath, 'r') as gene_features_file:
    gene_features_dict = json.load(gene_features_file)

# find which 'genes' are present in the gene_features_dict
gene_features_dict_genes = [f'{taxon}.{locus_tag}' for taxon in gene_features_dict.keys() for locus_tag in gene_features_dict[taxon]['genes'].keys()]

# write this list to a file
with open(f'{data_dir}{taxonomicID}_locus_tags_with_features.txt', 'w') as gene_features_genes_file:
    gene_features_genes_file.write('\n'.join(gene_features_dict_genes) + '\n')

## Download and Prune ASTRAL WoL Tree

In [9]:
%%bash -s "$taxonomicID" "$data_dir"
# remove existing files if they exist
rm -f $2"wol_metadata.tsv.xz" $2"wol_metadata.tsv" $2"wol_tree.nwk"
wget -nv https://biocore.github.io/wol/data/genomes/metadata.tsv.xz -O $2"wol_metadata.tsv.xz"
# this requires 7z
7z x $2"wol_metadata.tsv.xz" -o$2
# download the ASTRAL tree
wget -nv https://github.com/biocore/wol/raw/master/data/trees/astral/branch_length/cons/astral.cons.nwk -O $2"wol_tree.nwk"

2025-05-08 14:52:45 URL:https://biocore.github.io/wol/data/genomes/metadata.tsv.xz [971240/971240] -> "../data/wol_metadata.tsv.xz" [1]



7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF8,Utf16=on,HugeFiles=on,64 bits,128 CPUs AMD EPYC 7601 32-Core Processor                 (800F12),ASM,AES-NI)

Scanning the drive for archives:
1 file, 971240 bytes (949 KiB)

Extracting archive: ../data/wol_metadata.tsv.xz
--
Path = ../data/wol_metadata.tsv.xz
Type = xz
Physical Size = 971240
Method = LZMA2:26 CRC64
Streams = 1
Blocks = 1

Everything is Ok

Size:       6601710
Compressed: 971240


2025-05-08 14:52:46 URL:https://raw.githubusercontent.com/biocore/wol/master/data/trees/astral/branch_length/cons/astral.cons.nwk [407397/407397] -> "../data/wol_tree.nwk" [1]


In [10]:
# prune the ASTRAL WoL tree to only include the taxa for which we have gene data. This is our genome tree
# read in the tree

wol_metadata_filepath = f'{data_dir}wol_metadata.tsv'
wol_tree_filepath = f'{data_dir}wol_tree.nwk'
wol_tree = ete3.Tree(wol_tree_filepath, format=1)
wol_tree_leaf_names = wol_tree.get_leaf_names()

# read in the gene features json
json_filepath = f'{data_dir}{taxonomicID}_gene_features.json'
with open(json_filepath, 'r') as in_file:
    gene_features_dict = json.load(in_file)

# Read the metadata file from WOL as a dataframe
wol_df = pd.read_csv(wol_metadata_filepath, sep='\t', header=0)
print(f'Number of genomes in the metadata: {len(wol_df)}')
# change dtype of taxid column to str type
wol_df['taxid'] = wol_df['taxid'].astype(str)
# display first 5 rows
display(wol_df.head())
# check if every #genome in the wol_df maps to a unique assembly_accession value
assert wol_df['assembly_accession'].nunique() == wol_df['#genome'].nunique()
# smaller wol_df based on which taxid are in the gene_features_dict
wol_df = wol_df[wol_df['taxid'].isin(gene_features_dict.keys())]
print(f'Number of genomes after pruning based on taxid in gene_features_dict: {len(wol_df)}')

# gene_features_dict has taxid to acc_id mapping. wol_df has #genome to acc_id (`assembly_accession` column) mapping. Tree has #genome as leaf names
# we want to prune the trees and replace the leaf names with taxid values
acc_id_to_keep = set([i['acc_id'] for i in gene_features_dict.values()])
print(f'acc_id_to_keep looks like: {list(acc_id_to_keep)[:5]}')
# remove all other rows from the wol_df
wol_df = wol_df[wol_df['assembly_accession'].isin(acc_id_to_keep)]
print(f'Number of genomes in the pruned metadata: {len(wol_df)}')
genomes_to_keep = [i for i in wol_tree_leaf_names if i in wol_df['#genome'].values]

wol_tree.prune(genomes_to_keep, preserve_branch_length=True)
print(f'Number of genomes in the pruned tree: {len(wol_tree.get_leaf_names())}')

# create a dictionary mapping genome IDs to assembly accessions IDs
genome_to_accid_dict = wol_df.set_index('#genome')['assembly_accession'].to_dict()
# create a dictionary mapping assembly accessions to taxon IDs
accid_to_taxid_dict = {v['acc_id']: k for k, v in gene_features_dict.items()}
# create a dictionary mapping genome IDs to taxon IDs
genome_to_taxid_dict = {k: accid_to_taxid_dict[v]
                   for k, v in genome_to_accid_dict.items()}

# replace pruned tree leaf names with taxon IDs
for leaf in wol_tree.get_leaves():
    leaf.name = genome_to_taxid_dict[leaf.name]

Number of genomes in the metadata: 10575


,#genome,asm_name,assembly_accession,bioproject,biosample,wgs_master,seq_rel_date,submitter,ftp_path,img_id,...,coding_density,completeness,contamination,strain_heterogeneity,markers,5s_rrna,16s_rrna,23s_rrna,trnas,draft_quality
0,G000005825,ASM582v2,GCF_000005825.2,PRJNA224116,SAMN02603086,NaN,2010/12/15,"Center for Genomic Sciences, Allegheny-Singer ...",ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000...,646311908,...,85.144124,98.68,1.32,0.0,377,yes,yes,yes,20,high
1,G000006175,ASM617v2,GCF_000006175.1,PRJNA224116,SAMN00000040,NaN,2010/06/03,US DOE Joint Genome Institute (JGI-PGF),ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000...,646564549,...,80.167033,99.05,0.00,0.0,165,no,yes,yes,19,medium
2,G000006605,ASM660v1,GCF_000006605.1,PRJNA224116,SAMEA3283089,NaN,2005/06/27,Bielefeld Univ,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000...,637000085,...,89.378688,100.00,0.68,0.0,319,yes,yes,yes,20,high
3,G000006725,ASM672v1,GCF_000006725.1,PRJNA224116,SAMN02603773,NaN,2004/06/04,Sao Paulo state (Brazil) Consortium,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000...,637000348,...,82.592990,99.59,0.18,0.0,325,yes,yes,yes,20,high
4,G000006745,ASM674v1,GCF_000006745.1,PRJNA57623,SAMN02603969,NaN,2001/01/09,TIGR,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000...,637000333,...,86.533164,99.86,0.03,0.0,360,yes,yes,yes,20,high


Number of genomes after pruning based on taxid in gene_features_dict: 412
acc_id_to_keep looks like: ['GCF_000245735.1', 'GCA_001801365.1', 'GCF_000829415.1', 'GCF_000271305.1', 'GCF_001469215.1']
Number of genomes in the pruned metadata: 360
Number of genomes in the pruned tree: 360


In [11]:
# write the pruned tree to a newick file, with different variations in labelling for different program requirements
pruned_no_internal_wol_tree_filepath = f'{data_dir}{taxonomicID}_wol_tree_pruned_no_internal_labels.nwk'
pruned_angst_wol_tree_filepath = f'{data_dir}{taxonomicID}_wol_tree_pruned_angst.nwk'
pruned_internal_wol_tree_filepath = f'{data_dir}{taxonomicID}_wol_tree_pruned_with_internal_labels.nwk'

# write the pruned tree to a newick file without internal node names
wol_tree.write(outfile=pruned_no_internal_wol_tree_filepath, format=1, format_root_node=True, dist_formatter='%.10f')

# write a version of the tree with root length for AnGST.
# first make a copy of the tree
wol_tree.write(outfile=pruned_angst_wol_tree_filepath,  
               format=1, dist_formatter='%f', format_root_node=True)
# find the last ")" in this file and replace everything after it with ":0.0);".
# AnGST doesn't like the branch length at the root to be like 0.00000
with open(pruned_angst_wol_tree_filepath, "r") as fi:
    tree_str = fi.read()
    tree_str = tree_str[:tree_str.rfind(")")+1] + ":0.0);"
with open(pruned_angst_wol_tree_filepath, "w") as fo:
    fo.write(tree_str)

# traverse the tree in postorder and number the internal nodes as N1, N2, N3, etc.
# We don't skip a loop index if the node is a leaf. But we don't use the index for the leaf nodes, retaining the original name.
# We can use the enumerate function to get the index and the node at the same time.
internal_node_index = 1
for node in wol_tree.traverse(strategy="postorder"):
    if not node.is_leaf():
        node.name = f"N{internal_node_index}"
        internal_node_index += 1
# print the first 5 nodes of the tree
print(f"Node names look like: {
      [i for i in wol_tree.traverse(strategy='postorder')][:5]} ...")
# show the pruned tree leaf labels
print("New leaf names look like: ", wol_tree.get_leaf_names()[:10], "...")
# write this wol tree to a file. Branches are full floating point numbers
wol_tree.write(outfile=pruned_internal_wol_tree_filepath, 
               format=1, dist_formatter='%f', format_root_node=True)

Node names look like: [Tree node '1896966' (0x7f38b054c0b), Tree node '381306' (0x7f38afdb0b9), Tree node '246195' (0x7f38afe058b), Tree node '797473' (0x7f38afe0597), Tree node 'N1' (0x7f38afe057f)] ...
New leaf names look like:  ['1896966', '381306', '246195', '797473', '1766620', '555778', '1033802', '1304275', '1172194', '1579979'] ...


In [12]:
# how many taxa are in the tree and how many in the dictionary?
print("Number of taxa in the tree: ", len(wol_tree.get_leaves()), "Number of taxa in the dictionary: ", len(gene_features_dict))

Number of taxa in the tree:  360 Number of taxa in the dictionary:  853


## Extract gene trees of the subset data of interest

EggNOG 6.0 has one huge file with all the trees as well as (compressed) MSA. We need just the trees, and only for the NOGs that are of interest. Extract them.

In [13]:
%%bash -s "$taxonomicID" "$data_dir"
# use ripgrep to extract the trees for eggnog subset dataset (taxonomic ID should be in second column of TSV file). 
# Write column 1 and 3 to 2_all_trees.tsv
rg "\t$1\t" $2"e6.all_raw_trees_and_algs.tsv" | cut -f1,3 > $2$1"_all_trees.tsv"

In [14]:
# first we decide on which NOGs to keep. 

# we calculate the mean and in 0.25xSD around the mean, we include NOGs from the distribution of NOG sizes
# until the list of NOGs included, includes all the taxa to be included.

wol_tree = ete3.Tree(f"{data_dir}{taxonomicID}_wol_tree_pruned_no_internal_labels.nwk", format=1)
taxon_list = wol_tree.get_leaf_names()
taxon_set = set(taxon_list)

nog_members_tsv_filepath = f"{data_dir}{taxonomicID}_og2seqs_and_species.tsv"
nog_members_df = pd.read_csv(nog_members_tsv_filepath, sep='\t', header=None, names=['taxonomic_group',
                                                                                    'NOG',
                                                                                    '#taxa',
                                                                                    '#genes',
                                                                                    'taxa', 'genes'])


# pruning preparation
min_genes = 10  # minimum number of genes to retain a NOG
min_taxa = 30  # minimum number of taxa to retain a NOG
print("Min number of genes to retain a NOG: ", min_genes,
      "and min number of taxa to retain a NOG: ", min_taxa)

# taxa column in members_df contains a list of taxonomic IDs that are members of the NOG, as comma-seperated string
# Split the 'taxa' column by commas to create a list of taxa in each NOG, add it to a new column
nog_members_df['taxa'] = nog_members_df['taxa'].str.split(',')

# for the column 'genes' in nog_members_df, replace the periods with underscores
nog_members_df['genes'] = nog_members_df['genes'].str.replace('.', '_')
# there are mistakes in the 'gene' column, where some underscores are replaced with colon characters. Replace these with underscores
nog_members_df['genes'] = nog_members_df['genes'].str.replace(':', '_')
# Split the 'genes' column by commas to create a list of genes in each NOG
nog_members_df['genes'] = nog_members_df['genes'].str.split(',')

# replace the first underscore in 'genes' with a period, to get the taxon ID
nog_members_df['genes'] = nog_members_df['genes'].apply(
    lambda genes: [gene.replace('_', '.', 1) for gene in genes])
# Filter the list of genes based on the taxon_set
# Use the first part of an period-split to check if the taxonomic ID is in the taxon_list, and retain only those genes
nog_members_df['genes'] = nog_members_df['genes'].apply(
    lambda genes: [gene for gene in genes if gene.split('.')[0] in taxon_set])
# Create a new column that contains the number of genes to be retained in the gene tree
nog_members_df['#genes_to_keep'] = nog_members_df['genes'].str.len()
# remove rows where #genes_to_keep is less than 10 or more than max_genes
nog_members_df = nog_members_df[(nog_members_df['#genes_to_keep'] >= min_genes)]
# similarly, prune the taxa list column to only keep the taxa that are in the taxon_list
nog_members_df['taxa'] = nog_members_df['taxa'].apply(
    lambda taxa: [taxon for taxon in taxa if taxon in taxon_set])

# Sort the df by the absolute difference between #genes_to_keep and its mean
# This will give us the NOGs that are closest to the mean
mean_genes_to_keep = nog_members_df['#genes_to_keep'].mean()
nog_members_df['diff_from_mean'] = abs(
    nog_members_df['#genes_to_keep'] - mean_genes_to_keep)
nog_members_df = nog_members_df.sort_values(by='diff_from_mean')

# find std dev of #genes_to_keep in members_df
nog_members_df_std_dev = nog_members_df['#genes_to_keep'].std()
# display mean and std dev
print("Mean #genes_to_keep: ", nog_members_df['#genes_to_keep'].mean(
), "Std dev of #genes_to_keep: ", nog_members_df_std_dev)

# Flatten the list of taxa in the dataframe
all_taxa_in_df = set(
    [taxon for sublist in nog_members_df['taxa'].tolist() for taxon in sublist])

# Check if all taxa in taxon_list are represented in nog_members_df
if not set(taxon_list).issubset(all_taxa_in_df):
    print("Not all taxa in taxon_list are represented in nog_members_df")
    print("Taxa not represented: ", set(taxon_list) - all_taxa_in_df)
else:

    # retain only NOGs with at least min_taxa taxa
    nog_members_df = nog_members_df[nog_members_df['#taxa'] >= min_taxa]
    
    # check if the smallest n NOGs contain all the taxa in taxon_list, if not, increase n by 100 and repeat

    # start with a quarter of the std dev of #genes_to_keep in nog_members_df, rounded to closest higher 100
    n = int(np.ceil(nog_members_df_std_dev/4/100)*100)
    print("Starting with n = ", n,
          " as ~quarter of the std dev of #genes_to_keep in nog_members_df")
    while True:
        # make sure the loop doesn't run forever
        if n > len(nog_members_df):
            print("n is greater than the number of NOGs in the nog_members_df")
            break
        # get the smallest n NOGs
        nog_members_df_subset = nog_members_df.head(n)
        # create a set of all the taxa in the subset
        taxa_in_subset = set(
            [taxon for sublist in nog_members_df_subset['taxa'].tolist() for taxon in sublist])
        # check if the taxon_list is a subset of taxa_in_subset
        if set(taxon_list).issubset(taxa_in_subset):
            print("At n = ", n, ", taxon_list is a subset of taxa_in_subset, with total of ", len(
                taxa_in_subset), " taxa")
            break
        else:
            n += 100

# replace the nog_members_df with the subset, and rewrite the nog_members_df file
nog_members_df = nog_members_df_subset
# before writing out the nog_members_df, remove the columns that are not needed
nog_members_df = nog_members_df.drop(columns=['diff_from_mean'])
# also join the lists into comma-separated strings
nog_members_df['taxa'] = nog_members_df['taxa'].apply(lambda x: ','.join(x))
nog_members_df['genes'] = nog_members_df['genes'].apply(lambda x: ','.join(x))
nog_members_df.to_csv(f"{data_dir}{taxonomicID}_nog_members.tsv",
                  sep="\t", index=False, header=False)


# split the taxa and genes comma-separated strings back into lists
nog_members_df['taxa'] = nog_members_df['taxa'].str.split(',')
nog_members_df['genes'] = nog_members_df['genes'].str.split(',')

# display the last 5 rows of the nog_members_df
display(nog_members_df.tail())

# write out the list of taxa in this subset
with open(f"{data_dir}{taxonomicID}_subset_taxa.txt", "w") as taxa_fo:
    taxa_fo.write('\n'.join(taxon_list) + '\n')

Min number of genes to retain a NOG:  10 and min number of taxa to retain a NOG:  30
Mean #genes_to_keep:  84.05419908773813 Std dev of #genes_to_keep:  178.40979118790855
Starting with n =  100  as ~quarter of the std dev of #genes_to_keep in nog_members_df
At n =  1300 , taxon_list is a subset of taxa_in_subset, with total of  360  taxa


,taxonomic_group,NOG,#taxa,#genes,taxa,genes,#genes_to_keep
14341,1236,ER3QB,190,221,"[1294143, 1798802, 572477, 273526, 1441930, 40...","[105559.Nwat_2444, 1121939.L861_15835, 1178482...",55
14363,1236,ER3QZ,229,258,"[1926881, 230089, 910964, 69222, 1484157, 1681...","[1005058.UMN179_01181, 1006000.GKAS_02034, 105...",55
75558,1236,ESNNU,337,372,"[1736612, 216778, 273526, 871585, 1484157, 140...","[1006000.GKAS_04123, 1045855.DSC_01400, 111551...",55
17358,1236,ER6B2,254,301,"[743720, 717774, 491952, 1178482, 290398, 1445...","[1028989.PSCI_0675, 1051646.IX91_03105, 105164...",55
44651,1236,ERVXT,256,273,"[1736612, 43263, 1294143, 216778, 1535422, 179...","[1163408.UU9_10362, 1187848.A1QO_01760, 119576...",55


In [15]:
# Read in the TSV file with NOG to trees mapping
trees_df = pd.read_csv(f"{data_dir}{taxonomicID}_all_trees.tsv",
                       sep="\t", names=['NOG', 'tree'])
# Display length of the DataFrame
print("Length of trees_df: ", len(trees_df))
# Remove NOGs that are not in the nog_members_df
trees_df = trees_df[trees_df['NOG'].isin(nog_members_df['NOG'])]
# Display length of the new trees_df
print("Length of trees_df after filtering: ", len(trees_df))
# Overwrite the subset_trees.tsv file with the new trees
trees_df.to_csv(f"{data_dir}{taxonomicID}_subset_trees.tsv",
                sep="\t", index=False, header=False)

Length of trees_df:  103547
Length of trees_df after filtering:  1300


In [16]:
# read in wol_tree and make a list of taxa in the tree
wol_tree_filepath = f'{data_dir}{taxonomicID}_wol_tree_pruned_no_internal_labels.nwk'
wol_tree = ete3.Tree(wol_tree_filepath, format=1)
wol_tree_taxa = wol_tree.get_leaf_names()

# read in list of locus tags with features
all_genes_list_filepath = f'{data_dir}{taxonomicID}_locus_tags_with_features.txt'
with open(all_genes_list_filepath, 'r') as in_file:
    all_genes_list = in_file.read().splitlines()

# create a list of locus_tags that are in the wol_tree_taxa
all_locus_tags = [locus_tag for locus_tag in all_genes_list if locus_tag.split('.', 1)[0] in wol_tree_taxa]
print(f'Number of locus_tags considering the wol_tree_taxa: {len(all_locus_tags)}. Looks like: {all_locus_tags[:5]}')

# write this to a file
all_locus_tags_filepath = f'{data_dir}{taxonomicID}_subset_genes.txt'
with open(all_locus_tags_filepath, 'w') as out_file:
    out_file.write('\n'.join(all_locus_tags) + '\n')


Number of locus_tags considering the wol_tree_taxa: 1032613. Looks like: ['1005090.BAKON_001', '1005090.BAKON_002', '1005090.BAKON_003', '1005090.BAKON_004', '1005090.BAKON_005']


In [17]:
# prune trees
# read in the subset trees
subset_trees_filepath = f'{data_dir}{taxonomicID}_subset_trees.tsv'
subset_trees_df = pd.read_csv(subset_trees_filepath, sep='\t', header=None, names=['NOG', 'tree'])

# testing
# subset_trees_df = subset_trees_df.head(10)

# read in the subset genes
subset_genes_filepath = f'{data_dir}{taxonomicID}_subset_genes.txt'
with open(subset_genes_filepath, 'r') as in_file:
    subset_genes_list = in_file.read().splitlines()

def apply_prune_tree(args):
    """
    fn for multiprocessing pool to prune trees
    """
    index, row, subset_genes_list = args
    tree_newick = row['tree'].strip()
    # read in the tree
    tree = ete3.Tree(tree_newick, format=1)
    # get the leaf names
    leaf_names = tree.get_leaf_names()
    # find the genes that are in the tree
    leaves_to_keep = [leaf for leaf in leaf_names if leaf in subset_genes_list]
    # print(f'Number of leaves in tree: {len(leaf_names)}, Number of leaves to keep: {len(leaves_to_keep)}')
    # prune the tree
    if len(leaves_to_keep) <=4:
        return None
    elif len(leaves_to_keep) == len(leaf_names):
        return tree.write(format=1)
    else:
        tree.prune(leaves_to_keep)
        return tree.write(format=1)
    
# create a list of tuples to pass to the pool
args = [(index, row, subset_genes_list) for index, row in subset_trees_df.iterrows()]
max_proc = 2*mp.cpu_count()
with mp.Pool(max_proc) as pool:
    subset_trees_df['pruned_tree'] = list(pool.map(apply_prune_tree, args))

# remove rows where pruned_tree is None
print(f'Number of trees before pruning: {len(subset_trees_df)}')
subset_trees_df = subset_trees_df[subset_trees_df['pruned_tree'].notnull()]
print(f'Number of trees after pruning: {len(subset_trees_df)}')

# write the NOg: prune_tree mapping to a file
pruned_trees_filepath = f'{data_dir}{taxonomicID}_pruned_gene_trees.tsv'
subset_trees_df[['NOG', 'pruned_tree']].to_csv(pruned_trees_filepath, sep='\t', index=False, header=False)

Number of trees before pruning: 1300
Number of trees after pruning: 1300


In [18]:
# check if the taxa in wol tree, and taxa in set of all gene trees, are the exact same sets
# read in the rooted gene trees file
gene_trees_filepath = f'{data_dir}{taxonomicID}_pruned_gene_trees.tsv'
with open(gene_trees_filepath, 'r') as in_file:
    rooted_gene_trees = [i.split()[1].strip() for i in in_file.readlines()]

# create a set of all taxa in the gene trees
all_taxa_in_gene_trees = set()
for tree in rooted_gene_trees:
    gene_tree = ete3.Tree(tree, format=1)
    all_genes = gene_tree.get_leaf_names()
    all_taxa_in_gene_trees.update([gene.split('.', 1)[0] for gene in all_genes])

# read in the wol tree
wol_tree_no_internal_labels_filepath = f'{data_dir}{taxonomicID}_wol_tree_pruned_no_internal_labels.nwk'
wol_tree_with_internal_labels_filepath = f'{data_dir}{taxonomicID}_wol_tree_pruned_with_internal_labels.nwk'
wol_tree_angst_filepath = f'{data_dir}{taxonomicID}_wol_tree_pruned_angst.nwk'
wol_tree = ete3.Tree(wol_tree_no_internal_labels_filepath, format=1)
wol_tree_taxa = set(wol_tree.get_leaf_names())

# print which taxa are in wol tree but not in gene trees and vice versa
print(f'Taxa in wol tree but not in gene trees: {len(wol_tree_taxa - all_taxa_in_gene_trees)} \nand vice versa: {len(all_taxa_in_gene_trees - wol_tree_taxa)}')
print(f'They look like this: {list(wol_tree_taxa - all_taxa_in_gene_trees)[:5]} and {list(all_taxa_in_gene_trees - wol_tree_taxa)[:5]}')

# prune all the variants of the wol_tree to only include the taxa in the gene trees
wol_tree_taxa_to_keep = wol_tree_taxa.intersection(all_taxa_in_gene_trees)
print(f'Number of taxa to keep in the wol tree: {len(wol_tree_taxa_to_keep)}')
# prune the wol tree to only include the taxa in the gene trees
wol_tree.prune(wol_tree_taxa_to_keep)
wol_tree.write(outfile=wol_tree_no_internal_labels_filepath, format=1, format_root_node=True, dist_formatter='%.10f')

# write a version of the tree with root length for AnGST.
# first make a copy of the tree
wol_tree.write(outfile=wol_tree_angst_filepath,  
               format=1, dist_formatter='%f', format_root_node=True)
# find the last ")" in this file and replace everything after it with ":0.0);".
# AnGST doesn't like the branch length at the root to be like 0.00000
with open(wol_tree_angst_filepath, "r") as fi:
    tree_str = fi.read()
    tree_str = tree_str[:tree_str.rfind(")")+1] + ":0.0);"
with open(wol_tree_angst_filepath, "w") as fo:
    fo.write(tree_str)


# read in the wol tree with internal labels, prune it, and overwrite it
wol_tree = ete3.Tree(wol_tree_with_internal_labels_filepath, format=1)
wol_tree.prune(wol_tree_taxa_to_keep)
# relabel the internal node names
internal_node_index = 1
for node in wol_tree.traverse(strategy="postorder"):
    if not node.is_leaf():
        node.name = f"N{internal_node_index}"
        internal_node_index += 1
wol_tree.write(outfile=wol_tree_with_internal_labels_filepath, format=1, format_root_node=True, dist_formatter='%.10f')


Taxa in wol tree but not in gene trees: 1 
and vice versa: 0
They look like this: ['243277'] and []
Number of taxa to keep in the wol tree: 359


In [19]:
# write a new NOG members file for the subset of the NOGs and the genes in the subset
# read in original NOG members file
nog_members_tsv_filepath = f"{data_dir}{taxonomicID}_nog_members.tsv"
nog_members_df = pd.read_csv(nog_members_tsv_filepath, sep='\t', header=None, names=['taxonomic_group',
                                                                                    'NOG',
                                                                                    '#taxa',
                                                                                    '#genes',
                                                                                    'taxa', 'genes', '#genes_to_keep'])
# remove NOGs not in subset_trees_df
subset_trees_df = pd.read_csv(f"{data_dir}{taxonomicID}_pruned_gene_trees.tsv", sep="\t", header=None, names=['NOG', 'tree'])
nog_members_df = nog_members_df[nog_members_df['NOG'].isin(subset_trees_df['NOG'])]

# in `taxa` column keep only taxa that are in wol_tree_taxa_to_keep
nog_members_df['taxa'] = nog_members_df['taxa'].apply(
    lambda taxa: [taxon for taxon in taxa.split(',') if taxon in wol_tree_taxa_to_keep])
# in `genes` column keep only genes that are from taxa in wol_tree_taxa_to_keep
nog_members_df['genes'] = nog_members_df['genes'].apply(
    lambda genes: [gene for gene in genes.split(',') if gene.split('.', 1)[0] in wol_tree_taxa_to_keep])
# update the #taxa and #genes columns by counting the length of the lists in `taxa` and `genes` columns
nog_members_df['#taxa'] = nog_members_df['taxa'].apply(len)
nog_members_df['#genes'] = nog_members_df['genes'].apply(len)
# drop the #genes_to_keep column
nog_members_df = nog_members_df.drop(columns=['#genes_to_keep'])

# join the lists into comma-separated strings
nog_members_df['taxa'] = nog_members_df['taxa'].apply(lambda x: ','.join(x))
nog_members_df['genes'] = nog_members_df['genes'].apply(lambda x: ','.join(x))

# overwrite the NOG members file
nog_members_df.to_csv(f"{data_dir}{taxonomicID}_nog_members.tsv", sep="\t", index=False, header=False)


## Root all of these gene trees using MAD

The below command is what you can run and then wait to finish. It will be run in the background (because of `nohup` and `disown`) even if you lose connection to the server. This process can take in the order of hours to finish, for a list of trees in the order of 10k trees.

```
# cd 01-prepare_data
$ nohup python src/run_MAD_on_EggNOG_parallel.py -i ../data/1236_pruned_gene_trees.tsv -m ~/bin/mad/mad -n 100 > ../data/nohup_mad_eggnog_rooting_1236.log & disown
```

**NOTE**: Pruning took roughly 1.5 hours for 15k trees

## Download genome sequence files for each of the taxa in dataset

```bash
nohup python src/download_ncbi_genome_sequences.py -i ../data/1236_subset_taxa.txt  --to-download genome -a ../data/genome_data/1236_all_accession_ids.tsv > nohup_genome_dload.log & disown
```

In [20]:
# list all the files downloaded into a TSV file with columns genome_accession and fna_filepath
genome_data_dir = f'{data_dir}/genome_data/'
genome_data_dir_fullpath = os.path.abspath(genome_data_dir)
genome_data_files = [i for i in os.listdir(genome_data_dir) if i.endswith('.fna')]
# filename basenames are of the form {taxonID}_{GCA_xxxxxxx.n}_{genomeAssembly}_genomic.fna
# we want to extract the genome accession from this
genome_accession_filepath_tuples_list = [('_'.join(i.split('_')[1:3]), os.path.join(genome_data_dir_fullpath, i)) for i in genome_data_files]
genome_accessions_filepaths_df = pd.DataFrame(genome_accession_filepath_tuples_list, columns=['genome_accession', 'fna_filepath'])
genome_accessions_filepaths_df.to_csv(f'{data_dir}{taxonomicID}_genome_fna_filepaths.tsv', sep='\t', index=False, header=True)
print(f'Number of genome accessions: {len(genome_accessions_filepaths_df)}, and they look like:')
display(genome_accessions_filepaths_df.head())

Number of genome accessions: 1473, and they look like:


,genome_accession,fna_filepath
0,GCF_000953635.1,/root/work/projects/paper_comparative_study_of...
1,GCF_900461565.1,/root/work/projects/paper_comparative_study_of...
2,GCF_900450575.1,/root/work/projects/paper_comparative_study_of...
3,GCF_015476275.1,/root/work/projects/paper_comparative_study_of...
4,GCF_000953655.1,/root/work/projects/paper_comparative_study_of...


In [21]:
# read in this rooted pruned trees file, and replace the leaf labels
# such that taxID.geneID_geneID2 is replaced with taxID_geneID_geneID2

# function to replace first underscore with dot in leaf label
def replace_first_dot_with_underscore_in_leaf_label(tree_line):
    tree_ID, tree = tree_line.split('\t')
    gene_tree = ete3.Tree(tree, format=1)
    for leaf in gene_tree:
        leaf.name = leaf.name.replace('.', '_', 1)
    # add a branch length of 0.0 to the root node
    gene_tree.get_tree_root().dist = 0.0
    return tree_ID+'\t'+gene_tree.write(format=1, dist_formatter='%f', format_root_node=True)


# create a pool of workers
max_workers = 500
pool = mp.Pool(max_workers)

pruned_rooted_gene_trees_tsv_filepath = f'{data_dir}{taxonomicID}_pruned_gene_trees.tsv.rooted'
# open both the input and output file and apply the function to each line in the input file
with open(pruned_rooted_gene_trees_tsv_filepath) as infile, open(f"{pruned_rooted_gene_trees_tsv_filepath}.underscored", 'w') as outfile:
    # iterate over each line in the input file
    for line in infile:
        # apply the function to the line and write the result to the output file
        result = pool.apply_async(
            replace_first_dot_with_underscore_in_leaf_label, (line.strip(),))
        outfile.write(result.get() + '\n')

# close the pool
pool.close()
pool.join()


In [22]:
# create a TSV file with gene ID, start position, end position, strand, and genome accession ID
# for gene IDs in the gene features json for the subset of genes in the gene trees

def return_gene_features(args):
    """
    generator fn to return gene features for a taxon as a list
    """
    taxon, genes_of_taxon, taxon_gene_features_dict = args
    gene_features_lists = []
    for gene in genes_of_taxon:
        if gene in taxon_gene_features_dict['genes']:
            try:
                gene_features = taxon_gene_features_dict['genes'][gene]
                gene_features_lists.append([gene, gene_features['id'], 
                       gene_features['start'], gene_features['end'],
                       gene_features['strand'], gene_features['seqid'],
                       taxon_gene_features_dict['acc_id'], taxon])
            except KeyError:
                print(f'KeyError for gene {gene} in taxon {taxon}, with accesion {taxon_gene_features_dict["acc_id"]}')
    return gene_features_lists

with open(f'{data_dir}{taxonomicID}_gene_features.json', 'r') as in_file:
    gene_features_dict = json.load(in_file)

with open(f'{data_dir}{taxonomicID}_subset_taxa.txt', 'r') as in_file:
    subset_taxa_list = in_file.read().splitlines()

args = [(taxon, list(gene_features_dict[taxon]['genes'].keys()), gene_features_dict[taxon]) for taxon in subset_taxa_list]

max_workers = len(args)

gene_features_tsv_filepath = f'{data_dir}{taxonomicID}_gene_features.tsv'
header_list = ['locus_tag', 'gene_id', 'start', 'end', 'strand', 'seqid', 'genome_accession', 'taxon_id']

BUFFER_SIZE = 1000  # Keeping a buffer to write to file in chunks
# the buffer is important otherwise IOStream will be the bottleneck

def write_buffer(buffer, out_file):
    out_file.write('\n'.join(buffer) + '\n')
    buffer.clear()

with open(gene_features_tsv_filepath, 'w') as out_file:
    # write the header
    out_file.write('\t'.join(header_list) + '\n')
    # as you get the results, write them to the file instead of storing them in memory
    buffer = []
    with mp.Pool(max_workers) as pool:
        for result in pool.imap(return_gene_features, args):
            for gene_features in result:
                buffer.append('\t'.join(map(str, gene_features)))
                if len(buffer) >= BUFFER_SIZE:
                    write_buffer(buffer, out_file)
        if buffer:
            write_buffer(buffer, out_file)

## Create Presence-Absence (PA) matrices for GLOOME and Count

In [24]:
def nogs_pa_matrix(taxonomicID) -> None:
    '''
    use the members.tsv file of the NOGs from eggNOG, to prepare the pa_matrix
    '''
    import pandas as pd
    import numpy as np
    import os

    # read in the members.tsv file, but only the NOG, and the taxa columns (column 1 and 4)
    members_df_subset = pd.read_csv(f"{data_dir}/{taxonomicID}_nog_members.tsv", sep='\t', header=None,
                                    names=['NOG', 'taxa', 'genes'], usecols=[1, 4, 5])
    # split the taxa column into a list of taxa
    members_df_subset['taxa'] = members_df_subset['taxa'].apply(
        lambda x: x.split(','))
    # similar for genes column
    members_df_subset['genes'] = members_df_subset['genes'].apply(
        lambda x: x.split(','))

    # create pa matrix based on the presence-absence of the taxa in the NOGs
    # first create a list of all the taxa
    taxa = list(
        set([item for sublist in members_df_subset['taxa'].tolist() for item in sublist]))
    # sort the list
    taxa.sort()
    # display how the list looks like
    print("The taxa list looks like this: {}".format(taxa[:10]), "...")

    # create a dataframe with the NOGs as columns and the taxa as index
    # fill the dataframe with 0s as strings
    pa_df = pd.DataFrame(
        index=taxa, columns=members_df_subset['NOG'], dtype=str).fillna('0')

    # fill in the presence-absence matrix based on whether the taxa are present in the list of taxa for each NOG
    for index, row in members_df_subset.iterrows():
        for taxon in row['taxa']:
            # find how many ("_"-delimited) genes in the corresponding gene column, have the taxon as the first element
            # (i.e. the taxon is the first element of the gene name)
            pa_df.loc[taxon, row['NOG']] = str(
                int(len([gene for gene in row['genes'] if gene.split('.', 1)[0] == taxon])))

    # display the size of the matrix/df
    print("The size of the pa_df is: {}".format(pa_df.shape))

    # TSV file for COUNT, with NOGs in index and taxa as columns
    pa_df.T.to_csv(f"{data_dir}/{taxonomicID}_pa_matrix.tsv", sep='\t')

    # now binarize the matrix (i.e. if a NOG has at least one gene from a taxon, it gets a 1, otherwise 0)
    pa_df = pa_df.map(lambda x: '1' if x != '0' else '0')

    # Concatenate the columns into a single string per taxon and write out as FASTA file
    fasta_df = pd.DataFrame(pa_df.astype(str).apply(
        ''.join, axis=1), columns=['sequence'])
    fasta_df.index = '>' + fasta_df.index.str.strip()
    fasta_df.to_csv(
        f"{data_dir}{taxonomicID}_pa_matrix.fasta", sep='\n', header=False)

    return None


# run function
nogs_pa_matrix(taxonomicID)

The taxa list looks like this: ['1005057', '1005058', '1005090', '1006000', '1009858', '1026882', '1028989', '1033802', '1036674', '1045855'] ...
The size of the pa_df is: (359, 1300)


In [25]:
%%bash -s "$data_dir"
# extract the function profiles for the NOGs in our dataset from the eggnog6 database

# remove entries with no function profile
rg -v "\{\}" $1"/eggnog6/e6.func_profiles.json" > $1"/eggnog6/e6.func_profiles.available.json"
# get function profiles for nog members in our dataset
rg -f <(cut -f2 $1"/1236_nog_members.tsv" )  $1"/eggnog6/e6.func_profiles.available.json" > $1"/1236.func_profiles.json"